In [ ]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
parentdir = os.path.dirname(parentdir)
sys.path.insert(0,parentdir)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/D2FM')
module_path = '/content/drive/My Drive/D2FM'
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
!pip install pykalman

In [ ]:
#import models.ddfm

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from typing import Tuple
from tensorflow import keras
from tensorflow.keras import layers
from statsmodels.tsa.statespace.dynamic_factor_mq import DynamicFactorMQ as DFM
from sklearn.model_selection import ParameterGrid
from typing import Tuple
import numpy as np
from pykalman import KalmanFilter

In [ ]:
#from google.colab import files
#uploaded = files.upload()

In [ ]:
#from models.ddfm import DDFM
import warnings
warnings.filterwarnings("ignore")
import random
from sklearn.metrics import mean_squared_error as mse
from typing import Tuple
from sklearn.preprocessing import StandardScaler

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/D2FM/sfr_train.csv')
df_test=pd.read_csv('/content/drive/MyDrive/D2FM/sfr_test.csv')
df_train_idx=df_train.set_index("index")
df_test_idx=df_test.set_index("index")

In [ ]:
scaler = StandardScaler()
df_train_idx_scaled = scaler.fit_transform(df_train_idx)
df_test_idx_scaled= scaler.fit_transform(df_test_idx)

In [ ]:
df_test_idx.sort_index(inplace=True)
mean_test = df_test_idx.mean().values
sigma_test = df_test_idx.std().values
data_test = (df_test_idx - mean_test ) / sigma_test

In [ ]:
#@tf.function
def mse_missing(y_actual: tf.Tensor, y_predicted: tf.Tensor) -> tf.Tensor:
    """
    Custom loss (mse) for missing data.
    """
    mask = tf.where(tf.math.is_nan(y_actual), tf.zeros_like(y_actual), tf.ones_like(y_actual))
    y_actual_ = tf.where(tf.math.is_nan(y_actual), tf.zeros_like(y_actual), y_actual)
    y_predicted_ = tf.multiply(y_predicted, mask)
    return keras.losses.mean_squared_error(y_actual_, y_predicted_)


def convergence_checker(y_prev, y_now, y_actual):
    # TODO: Consider converting all to tensorflow
    loss_minus = mse(y_prev[~np.isnan(y_actual)], y_actual[~np.isnan(y_actual)])
    loss = mse(y_now[~np.isnan(y_actual)], y_actual[~np.isnan(y_actual)])
    return np.abs(loss - loss_minus) / loss_minus, loss

def convert_decoder_to_numpy(decoder: keras.Model, has_bias: bool, factor_oder: int,
                             structure_decoder: tuple = None) -> Tuple[np.ndarray, np.ndarray]:
    """
    Method to convert the decoder (a keras Model) to a numpy object
    Args:
        decoder: decoder, a keras Model
        has_bias: whether there is a bias term
        factor_oder: lag order for the common factors
        structure_decoder: the structure of the decoder, None for single layer

    Returns:
        weights and biases
    """
    if structure_decoder is None:
        if has_bias:
            ws, bs = decoder.get_layer(index=-1).get_weights()
        else:
            ws = decoder.get_layer(index=-1).get_weights()[0]
            bs = np.zeros(ws.shape[1])
        # observable equation
        if factor_oder == 2:
            emission = np.hstack((
                # bs.reshape(-1, 1),  # bias term
                ws.T,  # weight term
                np.zeros((ws.shape[1], ws.shape[0])),  # make zero lagged values
                np.identity(ws.shape[1])  # idio
            ))
        elif factor_oder == 1:
            emission = np.hstack((
                # bs.reshape(-1, 1),  # bias term
                ws.T,  # weight term
                np.identity(ws.shape[1])  # idio
            ))
        else:
            raise NotImplementedError("Only VAR(2) or VAR(1) for common factors at the moment.")
    else:
        NotImplementedError("Nonlinear decoder not available yet!")

    return bs, emission


def get_transition_params(f_t: np.ndarray, eps_t: np.ndarray, factor_oder: int, bool_no_miss: np.ndarray) -> Tuple[
    np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Method to calculate tran
    Args:
        f_t: common factors
        eps_t: idiosyncratic terms
        factor_oder: lag order for the common factors
        bool_no_miss: array to keep track of non-missing values
    Returns:
        autoregressive matrix, covariance matrix residuals, unconditional mean, unconditional variance, latent states
    """
    if factor_oder == 2:
        f_past = np.hstack((f_t[1:-1, :], f_t[:-2, :]))
        A_f = (np.linalg.pinv(f_past.T @ f_past) @ f_past.T @ f_t[2:, :]).T
    elif factor_oder == 1:
        f_past = f_t[:-1, :]
        A_f = (np.linalg.pinv(f_past.T @ f_past) @ f_past.T @ f_t[1:, :]).T
    else:
        raise NotImplementedError("Only VAR(2) or VAR(1) for common factors at the moment.")
    # get AR coeffs. from idiosyncratic
    A_eps, _, _ = get_idio(eps_t, bool_no_miss)
    # companion form x_t = [1, f_t, f_t_1, eps_t]
    if factor_oder == 2:
        # x_t = np.vstack((np.ones((1, f_past.shape[0] + 1)), f_t[1:, :].T, f_t[:-1, :].T, eps_t[1:, :].T))
        x_t = np.vstack((f_t[1:, :].T, f_t[:-1, :].T, eps_t[1:, :].T))
        # A = np.vstack((
        #     np.hstack((1, np.zeros(x_t.shape[0] - 1))).reshape(1, -1),  # intercept
        #     np.hstack(
        #         (np.zeros((A_f.shape[0], 1)), A_f, np.zeros((A_f.shape[0], x_t.shape[0] - 1 - A_f.shape[1])))),
        #     # VAR factors
        #     np.hstack((np.zeros((A_f.shape[0], 1)), np.identity(A_f.shape[0]),
        #                np.zeros((A_f.shape[0], x_t.shape[0] - 1 - A_f.shape[0])))),
        #     np.hstack((np.zeros((x_t.shape[0] - (A_f.shape[1] + 1), A_f.shape[1] + 1)), A_eps))  # AR 1 idio
        # ))
        A = np.vstack((
            np.hstack(
                (A_f, np.zeros((A_f.shape[0], eps_t.shape[1])))),  # VAR factors
            np.hstack((np.identity(A_f.shape[0]), np.zeros((A_f.shape[0], A_f.shape[0] + eps_t.shape[1])))),
            np.hstack((np.zeros((eps_t.shape[1], A_f.shape[1])), A_eps))  # AR 1 idio
        ))
    elif factor_oder == 1:
        # x_t = np.vstack((np.ones((1, f_past.shape[0] + 1)), f_t.T, eps_t.T))
        x_t = np.vstack((f_t.T, eps_t.T))
        # A = np.vstack((
        #     np.hstack((1, np.zeros(x_t.shape[0] - 1))).reshape(1, -1),  # intercept
        #     np.hstack(
        #         (np.zeros((A_f.shape[0], 1)), A_f, np.zeros((A_f.shape[0], x_t.shape[0] - 1 - A_f.shape[1])))),
        #     # VAR factors
        #     np.hstack((np.zeros((x_t.shape[0] - (A_f.shape[1] + 1), A_f.shape[1] + 1)), A_eps))  # AR 1 idio
        # ))
        A = np.vstack((
            np.hstack((A_f, np.zeros((A_f.shape[0], eps_t.shape[1])))),  # VAR factors
            np.hstack((np.zeros((eps_t.shape[1], A_f.shape[1])), A_eps))  # AR 1 idio
        ))
    else:
        raise NotImplementedError("")
    # error term matrix
    w_t = x_t[:, 1:] - A @ x_t[:, :-1]
    W = np.diag(np.diag(np.cov(w_t)))
    # Set to unconditional moments of x_0 = [1, f_0, f_0_1, eps_0]
    mu_0 = np.mean(x_t, axis=1)
    Σ_0 = np.cov(x_t)
    # zero correlation with idiosyncratic and diagonal covariance among them
    Σ_0[:A_f.shape[1], A_f.shape[1]:] = 0
    Σ_0[A_f.shape[1]:, :A_f.shape[1]] = 0
    Σ_0[A_f.shape[1]:, A_f.shape[1]:] = np.diag(np.diag(Σ_0[A_f.shape[1]:, A_f.shape[1]:]))
    return A, W, mu_0, Σ_0, x_t


def get_idio(eps: np.ndarray, idx_no_missings: np.ndarray, min_obs: int = 10) -> Tuple[
    np.ndarray, np.ndarray, np.ndarray]:
    """
    Method to get AR(1) statistics from eps.
    Args:
        eps: series of idiosyncratic AR(1)s
        idx_no_missings: array to keep track of non-missing values
        min_obs: minimum number of observations to estimate the statistics

    Returns:
        autoregressive coefficients, mean, standard deviation
    """
    # init params statistics
    phi = np.zeros((eps.shape[1], eps.shape[1]))
    mu_eps = np.zeros(eps.shape[1])
    std_eps = np.zeros(eps.shape[1])
    # loop over idios
    for j in range(eps.shape[1]):
        to_select = idx_no_missings[:, j]  # ~np.isnan(self.z_actual[:, j])
        to_select = np.hstack((np.array([False]), to_select[:-1] * to_select[1:]))
        if np.sum(to_select) > min_obs:
            this_eps = eps[to_select, j]
        else:
            raise ValueError("Not enough observation to estimate idio AR(1) parameters.")
            # this_eps = self.eps[:, j]
        mu_eps[j] = np.mean(this_eps)
        std_eps[j] = np.std(this_eps)
        cov1_eps = np.cov(this_eps[1:], this_eps[:-1])[0][1]
        phi[j, j] = cov1_eps / (std_eps[j] ** 2)
    return phi, mu_eps, std_eps


In [ ]:
class BaseModel:
    """
    An empty base class for the models.
    """

    def __int__(self, *args, **kwargs):
        super().__int__()

    def fit(self, **kwargs):
        """
        Fit method.
        """
        pass

    def predict(self, **kwargs):
        """
        Predict method
        """
        pass


In [ ]:
class StateSpace(BaseModel):
    """
    Base class for state-space models.
    """

    def __init__(self, mean_z: np.ndarray, sigma_z: np.ndarray, transition_params: dict, measurement_params: dict,
                 filter_type: str = "KalmanFilter"):
        """
        The init method will build the state space model according to the selected filter.
        Args:
            mean_z: mean of the measurement variable
            sigma_z: standard deviation of the measurement variable
            transition_params: parameters of the transition equation
            measurement_params: parameters of the measurement equation
            filter_type: the type of filter selected

        """
        super().__init__()
        self.mean_z = mean_z
        self.sigma_z = sigma_z
        # init parameters of the state space to None
        self.H = None
        self.R = None
        self.F = None
        self.Q = None
        self.predict = None
        self.filter = None
        if filter_type == "KalmanFilter":
            # build a linear gaussian state-space model
            self.build_lgss(transition_params, measurement_params)
        else:
            raise NotImplementedError("Only KalmanFilter is implemented at the moment.")

    def build_lgss(self, transition_params: dict, measurement_params: dict) -> None:
        """
        This method builds a linear gaussian state space model of the following form:
            measurement: z_t = H x_t + v_t; v_t ∼ N(0, R)
            transition: x_t = F x_t-1 + w_t; w_t ∼ N(0, Q)
        Args:
            transition_params: parameters of the transition equation
            measurement_params: parameters of the measurement equation

        Returns:
            None, it updates the class attributes.
        """

        self.H, self.R = measurement_params["H"], measurement_params["R"]
        self.F, self.Q = transition_params["F"], transition_params["Q"]
        self.filter_predict = KalmanFilter(transition_matrices=self.F, observation_matrices=self.H,
                                           transition_covariance=self.Q, observation_covariance=self.R)
        self.predict = self.predict_lgss
        self.filter = self.kalman_filter


    def kalman_filter(self, z: np.ndarray, standardize=False, do_em: bool = False) -> Tuple[
        np.ndarray, np.ndarray]:
        """
        This method implements the Kalman Filter.
            measurement: z_t = H x_t + v_t; v_t ∼ N(0, R)
            transition: x_t = F x_t-1 + w_t; w_t ∼ N(0, Q)
        Args:
            z: observable realised values
            standardize: whether to standardize the inputs or not

        Returns:
            filtered states and variance-covariance matrix
        """
        z_cpy = z.copy()
        if standardize:
            z_cpy = (z_cpy - self.mean_z) / self.sigma_z
        # make dimensions consistent
        if z_cpy.ndim == 1:
            z_cpy = np.reshape(z_cpy, (1, z_cpy.shape[0]))
        z_cpy = np.ma.array(z_cpy)
        z_cpy[np.isnan(z_cpy)] = np.ma.masked
        if do_em:
            (filtered_state_means, filtered_state_covariances) = self.filter_predict.em(z_cpy).filter(z_cpy)
        else:
            (filtered_state_means, filtered_state_covariances) = self.filter_predict.filter(z_cpy)
        return filtered_state_means, filtered_state_covariances


    def predict_lgss(self, x_hat_start: np.ndarray, sigma_x_hat_start: np.ndarray, steps_ahead: int = 1) -> dict:
        x_hat = x_hat_start
        sigma_x_hat = sigma_x_hat_start
        predicted_states = []
        #predicted_covariances = []
        for _ in range(steps_ahead):
          # Predict the next state
          x_hat_next = self.F @x_hat
          #sigma_x_hat_next = self.state_space_dict["transition"]["F"] @ sigma_x_hat @ self.state_space_dict["transition"]["F"].T + self.Q
          # Store the predictions
          predicted_states.append(x_hat_next)
          #predicted_covariances.append(sigma_x_hat_next)
          # Update current state and covariance for next iteration
          x_hat = x_hat_next
          #sigma_x_hat = sigma_x_hat_next
        #return {"states": np.array(predicted_states), "covariances": np.array(predicted_covariances)}
        return {"states": np.array(predicted_states)}

    def predict_measurement(self, future_states: np.ndarray) -> np.ndarray:
      """
      Predicts future observations given future states.
      Args:
        future_states: Future states (f_{t+k}) estimated by the state-space model.
      Returns:
        Predicted future observations (y_{t+k}).
      """
      predicted_observations = self.H @ future_states.T
      return predicted_observations.T

In [ ]:
dir(ddfm6.filter)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [ ]:
dir(ddfm6)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'autoencoder',
 'batch_norm',
 'batch_size',
 'bool_miss',
 'bool_no_miss',
 'build_inputs',
 'build_model',
 'build_state_space',
 'data',
 'data_mod',
 'data_mod_only_miss',
 'data_tmp',
 'decoder',
 'disp',
 'encoder',
 'epoch',
 'eps',
 'factor_oder',
 'factors',
 'factors_filtered',
 'filter',
 'filter_type',
 'fit',
 'initializer',
 'lags_input',
 'latents',
 'link',
 'loss_now',
 'max_iter',
 'mean_z',
 'optimizer',
 'pre_train',
 'predict',
 'predict_observations',
 'rng',
 'sigma_z',
 'state_space',
 'state_space_dict',
 'structure_decoder',
 'structure_encoder',
 'tolerance',
 'train',
 'use_bias',
 

In [ ]:
ddfm6.mean_z.shape


(118,)

In [ ]:
ddfm6.factors_filtered.shape

(577, 6)

In [ ]:
ddfm6.state_space_dict["measurement"]["R"].shape

(118, 118)

In [ ]:
ddfm6.state_space_dict["transition"]["Q"].shape

(124, 124)

In [ ]:
ddfm6.state_space_dict["transition"]["F"].shape

(124, 124)

In [ ]:
ddfm6.state_space_dict["measurement"]["H"].shape

(118, 124)

In [ ]:
class DDFM(BaseModel):
    """
    A class implementing Deep Dynamic Factor Models.
    """

    def __init__(self, data: pd.DataFrame, lags_input: int = 0, structure_encoder: tuple = (16, 4),
                 structure_decoder: tuple = None, use_bias: bool = True, factor_oder: int = 2, seed: int = 3,
                 batch_norm: bool = True, link: str = 'relu', learning_rate: float = 0.005,
                 optimizer: str = 'Adam', decay_learning_rate: bool = True,
                 epochs: int = 100, batch_size: int = 100, max_iter=200, tolerance: float = 0.0005,
                 disp: int = 10):
        """

        Args:
            data: input data used for model training
            lags_input: number of lags of the inputs on the encoder (default is 0, i.e. same inputs and outputs)
            structure_encoder: number of layers and neurons for the encoder
            structure_decoder: number of layers and neurons for the decoder (default is None, i.e. asymmetric
                autoencoder with one single decoder linear layer)
            use_bias: whether to use bias term in the last decoder layer
            factor_oder: number of lags in the transition equation for the dynamics of the common factors
            seed: seed to control randomness for replicability
            batch_norm: whether to add batch norm layers into the encoder
            link: the type of link/activation function
            learning_rate: the learning rate for the optimizer
            optimizer: the selected optimizer
            decay_learning_rate: whether to use a decaying learning rate
            epochs: number of epochs between MCMC iterations
            batch_size: the size of the batch
            max_iter: maximum number of iterations for the MCMC
            tolerance: the tolerance to stop iterations
            disp: display intermediate results every "disp" iterations of MCMC

        """
        super().__init__()
        # common factors
        self.factor_oder = factor_oder
        if factor_oder not in [1, 2]:
            raise ValueError('factor_oder must be 1 or 2')
        # z is the observable
        print("@Info - Note: Sorting data.")
        data.sort_index(inplace=True)
        self.mean_z = data.mean().values
        self.sigma_z = data.std().values
        self.data = (data - self.mean_z) / self.sigma_z
        # keep track of the missings locations
        self.bool_miss = self.data.isnull()[lags_input:].values
        self.bool_no_miss = self.bool_miss == False
        # create copies of the original data (needed for training and pre-training)
        self.data_mod_only_miss, self.data_mod, self.data_tmp = self.data.copy(), self.data.copy(), self.data.copy()
        self.z_actual = self.data[lags_input:].values
        # autoencoder structure
        self.lags_input = lags_input
        self.structure_encoder = structure_encoder
        self.structure_decoder = structure_decoder
        self.use_bias = use_bias
        self.batch_norm = batch_norm
        self.link = link
        # self.start_quarterly = start_quarterly
        if self.structure_decoder is None:
            self.filter_type = "KalmanFilter"
        else:
            self.filter_type = "ToBeDefined"
        # seed setting
        self.rng = np.random.RandomState(seed)
        self.initializer = tf.keras.initializers.GlorotNormal(seed=seed)
        # learning process
        self.batch_size = batch_size
        self.epoch = epochs
        self.max_iter = max_iter
        self.tolerance = tolerance
        self.disp = disp
        # optimizer
        if decay_learning_rate:
            learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(learning_rate, decay_steps=epochs,
                                                                           decay_rate=0.96, staircase=True)
        if optimizer == 'SGD':
            self.optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
        elif optimizer == 'Adam':
            self.optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
        else:
            raise KeyError("Optimizer must be SGD or Adam")
        # attributes to be populated
        self.autoencoder = None
        self.encoder = None
        self.decoder = None
        self.eps = None
        self.factors = None
        self.factors_filtered = None
        self.state_space = None
        self.state_space_dict = dict()
        self.latents = dict()
        self.loss_now=None

    def build_inputs(self, interpolate: bool = True) -> None:
        """
        Method to build the inputs of the model from the dataset.
        Args:
            interpolate: whether to interpolate or not the missing values

        Returns:
            None, it updates the data attributes of the class
        """

        # create dict with variables and their lagged values
        new_dict = {}
        for col_name in self.data_mod:
            new_dict[col_name] = self.data_mod[col_name]
            # create lagged Series
            for lag in range(self.lags_input):
                new_dict['%s_lag%d' % (col_name, lag + 1)] = self.data_mod[col_name].shift(lag + 1)
        # convert to dataframe
        self.data_tmp = pd.DataFrame(new_dict, index=self.data_mod.index)
        # drop initial nans
        self.data_tmp = self.data_tmp[self.lags_input:]
        # interpolate
        if interpolate and self.data_tmp.isna().sum().sum() > 0:
            # self._x_.interpolate(method='spline', limit_direction='forward', inplace=True, order=3)
            self.data_tmp.interpolate(method='spline', limit_direction='both', inplace=True, order=3)

    def build_model(self) -> None:
        """
        Method to build the keras model.
        Returns:
            None, it updates the attributes related to the autoencoder.
        """
        # encoder
        inputs_ = keras.Input(shape=(int((self.lags_input + 1) * self.data.shape[1]),))
        if len(self.structure_encoder) > 1:
            encoded = layers.Dense(self.structure_encoder[0], activation=self.link,
                                   bias_initializer='zeros', kernel_initializer=self.initializer)(inputs_)
            for j in self.structure_encoder[1:]:
                if self.batch_norm:
                    encoded = layers.BatchNormalization()(encoded)
                encoded = layers.Dense(j, activation=self.link,
                                       kernel_initializer=self.initializer,
                                       bias_initializer='zeros')(encoded)
        else:
            encoded = layers.Dense(self.structure_encoder[0], bias_initializer='zeros',
                                   kernel_initializer=self.initializer)(inputs_)

        self.encoder = keras.Model(inputs_, encoded)
        # decoder
        latent_inputs = keras.Input(shape=(self.structure_encoder[-1],))
        if self.structure_decoder:
            decoded = layers.Dense(self.structure_decoder[0], activation=self.link,
                                   kernel_initializer=self.initializer,
                                   bias_initializer='zeros')(latent_inputs)
            for j in self.structure_decoder[1:]:
                decoded = layers.Dense(j, activation=self.link, kernel_initializer=self.initializer,
                                       bias_initializer='zeros')(decoded)
            output_ = layers.Dense(self.data.shape[1], bias_initializer='zeros',
                                   kernel_initializer=self.initializer, use_bias=self.use_bias)(decoded)
        else:
            output_ = layers.Dense(self.data.shape[1], bias_initializer='zeros',
                                   kernel_initializer=self.initializer, use_bias=self.use_bias)(latent_inputs)
        self.decoder = keras.Model(latent_inputs, output_)
        outputs_ = self.decoder(self.encoder(inputs_))
        # autoencoder
        self.autoencoder = keras.Model(inputs_, outputs_)

    def pre_train(self, min_obs: int = 50, mult_epoch_pre: int = 1) -> None:
        """
        Method to carry out pre-training of the model.
        Args:
            min_obs: minimum number of observations for pre-training with no interpolation for missings
            mult_epoch_pre: coefficient to be multiplied to number of epochs to deliver to the total number of epochs
                for pre-training

        Returns:
            None, it updates the autoencoders attributes
        """
        # build inputs without interpolation
        self.build_inputs(interpolate=False)
        # check number of observations, and if not enough then interpolate
        if len(self.data_tmp.dropna()) >= min_obs:
            inpt_pre_train = self.data_tmp.dropna().values
            self.autoencoder.compile(optimizer=self.optimizer, loss='mse')
        else:
            self.build_inputs()
            inpt_pre_train = self.data_tmp.dropna().values
            self.autoencoder.compile(optimizer=self.optimizer, loss=mse_missing)
        # build output
        oupt_pre_train = self.data_tmp.dropna()[self.data.columns].values
        # fit (pre-train) autoencoder
        self.autoencoder.fit(inpt_pre_train, oupt_pre_train, epochs=self.epoch * mult_epoch_pre,
                             batch_size=self.batch_size,
                             verbose=0)

    def train(self) -> None:
        """
        Method to train a Deep Dynamic Factor Model (see Algorithm 1 from the paper.)
        Returns:
            None, it updates attributes.
        """
        # re-compile the autoencoder to re-init the optimizer and possibly change the objective
        self.autoencoder.compile(optimizer=self.optimizer, loss=mse_missing)
        # construct initial input data
        self.build_inputs()
        # make prediction
        prediction_iter = self.autoencoder.predict(self.data_tmp.values)
        # update missings
        self.data_mod_only_miss.values[self.lags_input:][self.bool_miss] = prediction_iter[self.bool_miss]
        # get idio
        self.eps = self.data_tmp[self.data.columns].values - prediction_iter
        # init counters
        iter = 0
        not_converged = True
        loss_now = None
        # start MCMC
        while not_converged and iter < self.max_iter:
            # get idio distr
            phi, mu_eps, std_eps = get_idio(self.eps, self.bool_no_miss)
            # subtract conditional AR-idio mean from x
            self.data_mod[self.lags_input + 1:] = self.data_mod_only_miss[self.lags_input + 1:] - self.eps[:-1, :] @ phi
            # for first observations set to 0 the idio
            self.data_mod[:self.lags_input + 1] = self.data_mod_only_miss[:self.lags_input + 1]
            # gen data_tmp from filtered inputs (self.data_mod above)
            self.build_inputs()
            # gen MC samples for idio (dims = Sim x T x D)
            eps_draws = self.rng.multivariate_normal(mu_eps, np.diag(std_eps), (self.epoch, self.data_tmp.shape[0]))
            # init noisy inputs (dims = Sim x T x D_with_lags)
            x_sim_den = np.zeros((eps_draws.shape[0], eps_draws.shape[1], eps_draws.shape[2] * (self.lags_input + 1)))
            # loop over them (MC step)
            for i in range(self.epoch):
                x_sim_den[i, :, :] = self.data_tmp.copy()
                # corrupt input data, only current observations
                x_sim_den[i, :, :eps_draws[i, :, :].shape[1]] = x_sim_den[i, :,
                                                                :eps_draws[i, :, :].shape[1]] - eps_draws[i, :, :]
                # fit autoencoder
                self.autoencoder.fit(x_sim_den[i, :, :], self.z_actual, epochs=1, batch_size=self.batch_size, verbose=0)
            # update factors: average over all predictions from the MC samples
            self.factors = np.array([self.encoder(x_sim_den[i, :, :]) for i in range(x_sim_den.shape[0])])
            # check convergence
            prediction_iter = np.mean(np.array([self.decoder(self.factors[i, :, :]) for i in range(self.factors.shape[0]
                                                                                                   )]), axis=0)
            if iter > 1:
                delta, self.loss_now = convergence_checker(prediction_prev_iter, prediction_iter, self.z_actual)
                #loss_now = loss_now
                if iter % self.disp == 0:
                    return self.loss_now
                    print(f'@Info: iteration: {iter} - new loss: {self.loss_now} - delta: {delta}')
                if delta < self.tolerance:
                    not_converged = False
                    print(f'@Info: Convergence achieved in {iter} iterations - new loss: {self.loss_now} - delta: {delta} < {self.tolerance}')
            # store previous prediction to monitor convergence
            prediction_prev_iter = prediction_iter.copy()
            # update missings
            self.data_mod_only_miss.values[self.lags_input:][self.bool_miss] = prediction_iter[self.bool_miss]
            # update idio
            self.eps = self.data_mod_only_miss.values[self.lags_input:] - prediction_iter
            iter += 1
        if not_converged:
            print("@Info: Convergence not achieved within the maximum number of iteration!")

    def build_state_space(self) -> None:
        """
        Method to build the state space model from the autoencoder.
            measurement: z_t = H x_t + v_t; v_t ∼ N(0, R)
            transition: x_t = F x_t-1 + w_t; w_t ∼ N(0, Q)
        Returns:
            None, it updates the class attributes.
        """
        # extract common factors
        f_t = np.mean(self.factors, axis=0)
        # idio components
        eps_t = self.eps
        # get params from decoder (measurement equation)
        bs, H = convert_decoder_to_numpy(self.decoder, self.use_bias, self.factor_oder,
                                         structure_decoder=self.structure_decoder)
        # modify mean with the bias term
        self.mean_z = self.mean_z + bs * self.sigma_z
        # get transition equation params
        F, Q, mu_0, sigma_0, x_t = get_transition_params(f_t, eps_t, factor_oder=self.factor_oder,
                                                         bool_no_miss=self.bool_no_miss)
        # insert in dictionary
        self.state_space_dict["transition"] = dict()
        self.state_space_dict["transition"]["F"] = F
        self.state_space_dict["transition"]["Q"] = Q
        self.state_space_dict["transition"]["mu_0"] = mu_0
        self.state_space_dict["transition"]["Σ_0"] = sigma_0
        self.latents["ae_states"] = x_t
        # we set this to a small number, but we could cross-validate to control the signal-to-noise ratio
        R = np.eye(eps_t.shape[1]) * 1e-15
        # H = None
        self.state_space_dict["measurement"] = dict()
        self.state_space_dict["measurement"]["H"] = H
        self.state_space_dict["measurement"]["R"] = R
        self.state_space = StateSpace(self.mean_z, self.sigma_z,
                                      self.state_space_dict["transition"], self.state_space_dict["measurement"],
                                      filter_type=self.filter_type)

    def fit(self):
        """
        Method to fit the Deep Dynamic Factor Model.
        Returns:
            None, it updates the class attributes.
        """
        self.build_model()
        self.pre_train()
        self.train()
        self.build_state_space()
        # get filtered factors
        self.latents["filtered"], self.latents["sigma_kf"] = self.filter(self.data.values)
        self.factors_filtered = self.latents["filtered"][:, 1:self.structure_encoder[-1] + 1]
    def get_last_state(self):
        """
        Retrieves the last state and its variance-covariance matrix from the fitted model.
        Returns:
        tuple: A tuple containing the last state and its variance-covariance matrix.
        """
        last_state = self.latents["filtered"][-1]
        last_state_var_cov_matrix = self.latents["sigma_kf"][-1]
        return last_state, last_state_var_cov_matrix


    def filter(self, z_t: np.ndarray, standardize: bool = False) -> Tuple[
        np.ndarray, np.ndarray]:
        """
        Method to carry out the filtering in state-space.
        Args:
            z_t: observable realised values
            standardize: whether to standardize the inputs or not

        Returns:
            filtered states and variance-covariance matrix
        """
        return self.state_space.filter(z_t, standardize=standardize)


    def predict(self, x_hat_start: np.ndarray, sigma_x_hat_start: np.ndarray, steps_ahead: int = 1) -> dict:
        """
        Method to carry out the prediction in state-space.
        Args:
            x_hat_start: starting values for the state mean
            sigma_x_hat_start: starting values for the state variance covariance matrix
            steps_ahead: number of steps ahead

        Returns:
            A dictionary with predicted states and measurement mean and variances.
        """
        return self.state_space.predict(x_hat_start, sigma_x_hat_start, steps_ahead=steps_ahead)

    def predict_observations(self, x_hat_start: np.ndarray, sigma_x_hat_start: np.ndarray, steps_ahead: int = 1) -> np.ndarray:
      """
      Predicts future observations (y_{t+k}) using the deep dynamic factor model.
      Args:
        x_hat_start: Starting values for the state mean.
        sigma_x_hat_start: Starting values for the state variance covariance matrix.
        steps_ahead: Number of steps ahead.
      Returns:
        Predicted future observations (y_{t+k}).
      """
      # Predict future states (f_{t+k})
      future_states = self.predict(x_hat_start, sigma_x_hat_start, steps_ahead)['states']
      # Predict future observations (y_{t+k})
      future_observations = self.state_space.predict_measurement(future_states)
      return future_observations






In [ ]:
#ddfm6.state_space_dict["transition"]["Σ_0"].shape

(124, 124)

# Para 7 factores:

In [ ]:
r_true=7
structure_encoders = [
    (r_true * 4, r_true * 2, r_true),
    (r_true * 6, r_true * 4, r_true),
    (r_true * 12, r_true * 6, r_true)
]

lags_inputs = list(range(2))

model_info = []

# Iterate over both True and False values for use_bias
for use_bias_setting in [True, False]:
    # Iterate over each structure_encoder configuration
    for structure_encoder in structure_encoders:
        # Iterate over each lag value
        for lags_input in lags_inputs:
            # Initialize the DDFM with the current configuration and use_bias setting
            ddfm = DDFM(df_train_idx, lags_input=lags_input, structure_encoder=structure_encoder,
                        optimizer='Adam', factor_oder=1, use_bias=use_bias_setting, link='tanh',
                        epochs=100, max_iter=1000)

            # Fit the model
            ddfm.fit()

            # Store the model, its configuration, and final loss
            final_loss = ddfm.loss_now
            model_info.append((ddfm, structure_encoder, lags_input, use_bias_setting, final_loss))

# Print the information for each model
for model, encoder_config, lag, use_bias_setting, final_loss in model_info:
    print(f"Model with encoder {encoder_config}, lag {lag}, use_bias {use_bias_setting} - Final Loss: {final_loss}")

@Info - Note: Sorting data.
19/19 [==============================] - 0s 3ms/step
@Info: Convergence achieved in 8 iterations - new loss: 0.568943095022181 - delta: 0.000360916890050795 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 8 iterations - new loss: 0.5632790321861864 - delta: 2.0773473356622294e-05 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 7 iterations - new loss: 0.5676663599891031 - delta: 0.00027047013742516676 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 8 iterations - new loss: 0.5611486675911035 - delta: 0.00042446792035540976 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 3ms/step
@Info: Convergence achieved in 8 iterations - new loss: 0.5654334640322526 - delta: 5.01459603299358e-05 < 0.0005
@Info - Note: Sortin

In [ ]:
# Initialize variables to store the minimum loss and associated configuration.
min_loss = float('inf')
min_loss_config = None

# Iterate over the stored models to find the one with the minimum loss.
for model, encoder_config, lag, use_bias_setting, final_loss in model_info:
    if final_loss < min_loss:
        min_loss = final_loss
        min_loss_config = (encoder_config, lag, use_bias_setting)

# Print the configuration with the minimum loss.
if min_loss_config is not None:
    print(f"Minimum Loss: {min_loss} with encoder {min_loss_config[0]}, lag {min_loss_config[1]}, and use_bias {min_loss_config[2]}")
else:
    print("No models found or no valid loss values.")


Minimum Loss: 0.5601520308495807 with encoder (84, 42, 7), lag 1, and use_bias False


In [ ]:
ddfm7 = DDFM(df_train_idx, structure_encoder=(84, 42, 7), lags_input=1, factor_oder=1,
                             use_bias=False, link='tanh', max_iter=1000)
ddfm7.fit()

@Info - Note: Sorting data.
18/18 [==============================] - 0s 4ms/step
@Info: Convergence achieved in 6 iterations - new loss: 0.5601794409845514 - delta: 0.00043359827501088005 < 0.0005


In [ ]:
last_state, last_state_var_cov_matrix = ddfm7.get_last_state()

In [ ]:
f_hat=ddfm7.state_space_dict["transition"]["F"]@x_hat_start
f_hat.shape

(125,)

In [ ]:
steps_ahead =12  # for example, predict 5 steps ahead
predictions7 = ddfm7.predict(last_state, last_state_var_cov_matrix, steps_ahead)
#predictions7['states'].shape
y_t_k_7=ddfm7.predict_observations(last_state,last_state_var_cov_matrix , steps_ahead)
df_test_first_12_rows = data_test.iloc[:12, :]
test_array12 = df_test_first_12_rows.to_numpy()
mse_row12=np.mean((test_array12 - y_t_k_7)**2, axis=1)
mse_12=np.mean(mse_row12)
mae_12 = np.mean(np.abs(test_array12 - y_t_k_7))
rmsfe_12 = np.sqrt(np.mean((test_array12 - y_t_k_7) ** 2))
print("Mean Squared Error:", mse_12)
print("Mean Absolute Error:", mae_12)
print("Root Mean Squared Forecast Error:", rmsfe_12)

Mean Squared Error: 3.2279855049036486
Mean Absolute Error: 1.2606368648611543
Root Mean Squared Forecast Error: 1.7966595406207735


In [ ]:
steps_ahead = df_test_idx.shape[0]  # for example, predict 5 steps ahead
predictions7 = ddfm7.predict(last_state, last_state_var_cov_matrix, steps_ahead)
#predictions7['states'].shape
y_t_k_7=ddfm7.predict_observations(last_state, last_state_var_cov_matrix, steps_ahead)
#df_test_first_12_rows = df_test_idx.iloc[:12, :]
test_array = data_test.to_numpy()
mse_row=np.mean((test_array - y_t_k_7)**2, axis=1)
mse_145=np.mean(mse_row)
mae_145 = np.mean(np.abs(test_array - y_t_k_7))
rmsfe_145 = np.sqrt(np.mean((test_array - y_t_k_7) ** 2))
print("Mean Squared Error:", mse_145)
print("Mean Absolute Error:", mae_145)
print("Root Mean Squared Forecast Error:", rmsfe_145)

Mean Squared Error: 0.9740722982081937
Mean Absolute Error: 0.6888614785125942
Root Mean Squared Forecast Error: 0.9869510110477591


# Para 6 factores:

In [ ]:
r = 6
structure_encoders = [(r * 6, r * 4, r * 2, r),
                      (r * 12, r * 6, r * 4, r)]

lags_inputs = list(range(2))

model_info = []

# Iterate over both True and False values for use_bias
for use_bias_setting in [True, False]:
    # Iterate over each structure_encoder configuration
    for structure_encoder in structure_encoders:
        # Iterate over each lag value
        for lags_input in lags_inputs:
            # Initialize the DDFM with the current configuration and use_bias setting
            ddfm = DDFM(df_train_idx, lags_input=lags_input, structure_encoder=structure_encoder,
                        optimizer='Adam', factor_oder=1, use_bias=use_bias_setting, link='tanh',
                        epochs=100, max_iter=1000)

            # Fit the model
            ddfm.fit()

            # Store the model, its configuration, and final loss
            final_loss = ddfm.loss_now
            model_info.append((ddfm, structure_encoder, lags_input, use_bias_setting, final_loss))

# Print the information for each model
for model, encoder_config, lag, use_bias_setting, final_loss in model_info:
    print(f"Model with encoder {encoder_config}, lag {lag}, use_bias {use_bias_setting} - Final Loss: {final_loss}")

@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 4 iterations - new loss: 0.6044490276205244 - delta: 0.0003656000110053137 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 6 iterations - new loss: 0.5924190376465651 - delta: 0.00047010963246637725 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 4 iterations - new loss: 0.5991136372167796 - delta: 0.00018932438242943 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 5 iterations - new loss: 0.5905504150709575 - delta: 0.00038853532190897665 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 7 iterations - new loss: 0.6019140286258788 - delta: 0.0003068920457274224 < 0.0005
@Info - Note: Sortin

In [ ]:
# Initialize variables to store the minimum loss and associated configuration.
min_loss = float('inf')
min_loss_config = None

# Iterate over the stored models to find the one with the minimum loss.
for model, encoder_config, lag, use_bias_setting, final_loss in model_info:
    if final_loss < min_loss:
        min_loss = final_loss
        min_loss_config = (encoder_config, lag, use_bias_setting)

# Print the configuration with the minimum loss.
if min_loss_config is not None:
    print(f"Minimum Loss: {min_loss} with encoder {min_loss_config[0]}, lag {min_loss_config[1]}, and use_bias {min_loss_config[2]}")
else:
    print("No models found or no valid loss values.")


Minimum Loss: 0.5898424215737922 with encoder (72, 36, 24, 6), lag 1, and use_bias False


In [ ]:
ddfm6 = DDFM(df_train_idx, structure_encoder=(72, 36, 6), lags_input=1, factor_oder=1,
                             use_bias=False, link='tanh', max_iter=1000)
ddfm6.fit()

@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 7 iterations - new loss: 0.5897091840144462 - delta: 0.00026435753202212536 < 0.0005


In [ ]:
last_state, last_state_var_cov_matrix = ddfm6.get_last_state()

In [ ]:
last_state.shape

(124,)

In [ ]:
steps_ahead = df_test_idx.shape[0]  # for example, predict 5 steps ahead
predictions6 = ddfm6.predict(last_state, last_state_var_cov_matrix, steps_ahead)
#predictions7['states'].shape
y_t_k_6=ddfm6.predict_observations(x_hat_start6, sigma_x_hat_start6, steps_ahead)
#df_test_first_12_rows = df_test_idx.iloc[:12, :]
test_array = data_test.to_numpy()
mse_row=np.mean((test_array - y_t_k_6)**2, axis=1)
mse_145=np.mean(mse_row)
mae_145 = np.mean(np.abs(test_array - y_t_k_6))
rmsfe_145 = np.sqrt(np.mean((test_array - y_t_k_6) ** 2))
print("Mean Squared Error:", mse_145)
print("Mean Absolute Error:", mae_145)
print("Root Mean Squared Forecast Error:", rmsfe_145)

Mean Squared Error: 1.0300044974802698
Mean Absolute Error: 0.703399646960847
Root Mean Squared Forecast Error: 1.014891372256297


In [ ]:
#n = ddfm.state_space.F.shape[0]
#x_hat_start = np.zeros((1, n))
#sigma_x_hat_start = np.eye(ddfm.state_space.F.shape[0])
#x_hat_start.shape

In [ ]:
#dir(ddfm)

In [ ]:
f_hat=ddfm6.state_space_dict["transition"]["F"]@x_hat_start6
f_hat.shape

(124,)

### factores filtrados

In [ ]:
ff6=ddfm6.factors_filtered
ff6.shape

(577, 6)

In [ ]:
f_hat_ddfm_idx6 = pd.DataFrame(ff6, columns=["f1", "f2", "f3", "f4", "f5", "f6"])

In [ ]:
f_hat_ddfm_idx6.head()

,f1,f2,f3,f4,f5,f6
0,-0.512616,0.008194,0.166183,0.624644,0.487667,-0.904841
1,0.217684,0.307360,-0.116110,0.154630,-0.606569,0.121623
2,-0.393536,0.412020,0.033630,-0.017483,-0.588406,0.550374
3,0.082301,-0.095549,0.023001,-0.004909,-0.395127,0.356847
4,-0.170490,0.375945,0.173957,-0.112834,0.075884,0.209188


In [ ]:
f_hat_ddfm_idx6.index = df_train_idx.index
f_hat_ddfm6=f_hat_ddfm_idx6.reset_index()

In [ ]:
f_hat_ddfm6.to_csv('/content/drive/MyDrive/D2FM/f_hat_ddfm6.csv', index=False)